In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pymongo import MongoClient
import numpy as np

# List of all 50 states
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", 
    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", 
    "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", 
    "Nevada", "New-Hampshire", "New-Jersey", "New-Mexico", "New-York", "North-Carolina", "North-Dakota", "Ohio", 
    "Oklahoma", "Oregon", "Pennsylvania", "Rhode-Island", "South-Carolina", "South-Dakota", "Tennessee", "Texas", 
    "Utah", "Vermont", "Virginia", "Washington", "West-Virginia", "Wisconsin", "Wyoming"
]

# Initialize an empty list to hold all the data
all_data = []

# Loop through each state
for state in states:
    url = f"https://www.redfin.com/state/{state}"  # Update the URL with the current state
    
    try:
        # Send a GET request to the webpage
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Locate the table based on its class name
        table = soup.find('table', {'class': 'filterableTable table-styles-basic'})
        
        # Skip if the table is not found
        if not table:
            print(f"No data found for {state}")
            continue

        # Extract the rows from the table body
        rows = table.find('tbody').find_all('tr')

        # Parse and extract data from each row
        for row in rows:
            columns = row.find_all('td')
            row_data = [col.get_text(strip=True) for col in columns]
            row_data.append(state)  # Add the state as an additional column
            all_data.append(row_data)
        
        print(f"Data successfully extracted for {state}")
    except Exception as e:
        print(f"Error extracting data for {state}: {e}")

# Check if any data was collected
if all_data:
    # Define the column names
    column_names = ["city_state", "avg.list price", "avg.price/sqft", "avg.days on market", "state"]

    # Create a pandas DataFrame
    df = pd.DataFrame(all_data, columns=column_names)

    # Split the "city_state" column into "city" and "state"
    df[['city', 'state']] = df['city_state'].str.split(',', expand=True)
    df['city'] = df['city'].str.strip()  # Remove any leading/trailing spaces in city names
    df['state'] = df['state'].str.strip()  # Remove any leading/trailing spaces in state names

    # Drop rows where any of these columns have NaN (i.e., non-numeric values)
    df = df.dropna(subset=['avg.list price', 'avg.price/sqft', 'avg.days on market'])


Data successfully extracted for Alabama
Data successfully extracted for Alaska
Data successfully extracted for Arizona
Data successfully extracted for Arkansas
Data successfully extracted for California
Data successfully extracted for Colorado
Data successfully extracted for Connecticut
Data successfully extracted for Delaware
Data successfully extracted for Florida
Data successfully extracted for Georgia
Data successfully extracted for Hawaii
Data successfully extracted for Idaho
Data successfully extracted for Illinois
Data successfully extracted for Indiana
Data successfully extracted for Iowa
Data successfully extracted for Kansas
Data successfully extracted for Kentucky
Data successfully extracted for Louisiana
Data successfully extracted for Maine
Data successfully extracted for Maryland
Data successfully extracted for Massachusetts
Data successfully extracted for Michigan
Data successfully extracted for Minnesota
Data successfully extracted for Mississippi
Data successfully extr

In [38]:
df.head()

,city_state,avg.list price,avg.price/sqft,avg.days on market,state,city
0,"1.Huntsville, AL","$375,000",$161,91,AL,1.Huntsville
1,"2.Birmingham, AL","$165,000",$89,82,AL,2.Birmingham
2,"3.Mobile, AL","$249,584",$124,85,AL,3.Mobile
3,"4.Madison, AL","$549,900",$177,109,AL,4.Madison
4,"5.Montgomery, AL","$190,000",$108,82,AL,5.Montgomery


In [40]:

# Rearrange the columns in the desired order
df = df[['state', 'city', 'avg.list price', 'avg.price/sqft', 'avg.days on market']]
df.head(100)

,state,city,avg.list price,avg.price/sqft,avg.days on market
0,AL,1.Huntsville,"$375,000",$161,91
1,AL,2.Birmingham,"$165,000",$89,82
2,AL,3.Mobile,"$249,584",$124,85
3,AL,4.Madison,"$549,900",$177,109
4,AL,5.Montgomery,"$190,000",$108,82
...,...,...,...,...,...
95,CA,16.Rancho Cucamonga,"$859,999",$454,52
96,CA,17.Fullerton,"$987,450",$569,57
97,CA,18.Huntington Beach,"$1,494,500",$758,59
98,CA,19.Sunnyvale,"$1,648,236","$1,209",54


In [42]:
 # Save the DataFrame to a CSV file
csv_filename = "redfin_all_states_data.csv"
if not df.empty:  # Assuming you're checking if the DataFrame has data
    df.to_csv(csv_filename, index=False)
    print(f"Data successfully saved to {csv_filename}")
else:
    print("No data was collected.")


Data successfully saved to redfin_all_states_data.csv


In [31]:
import pandas as pd

# Read the CSV file containing data for all 50 states
df_2 = pd.read_csv("redfin_all_states_data.csv")  # Replace with your actual file path

# Group the DataFrame by 'state'
grouped = df.groupby('state')

# Create a dictionary where each key is a state and the value is the corresponding DataFrame
state_dfs = {state: data for state, data in grouped}

states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# Assign the DataFrame for each state
for state in states:
    globals()[f'{state.lower()}_df'] = state_dfs[state]


In [33]:
# List all variable names in the global namespace that start with the state abbreviation (e.g., 'al_df', 'ak_df', ...)
state_dfs_list = [var for var in globals() if var.endswith('_df')]

# Display the list of all DataFrame variable names
print(state_dfs_list)

['al_df', 'ak_df', 'az_df', 'ar_df', 'ca_df', 'co_df', 'ct_df', 'de_df', 'fl_df', 'ga_df', 'hi_df', 'id_df', 'il_df', 'in_df', 'ia_df', 'ks_df', 'ky_df', 'la_df', 'me_df', 'md_df', 'ma_df', 'mi_df', 'mn_df', 'ms_df', 'mo_df', 'mt_df', 'ne_df', 'nv_df', 'nh_df', 'nj_df', 'nm_df', 'ny_df', 'nc_df', 'nd_df', 'oh_df', 'ok_df', 'or_df', 'pa_df', 'ri_df', 'sc_df', 'sd_df', 'tn_df', 'tx_df', 'ut_df', 'vt_df', 'va_df', 'wa_df', 'wv_df', 'wi_df', 'wy_df']
